In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
import time

In [17]:
from bs4 import BeautifulSoup

In [18]:
import requests
import numpy
import pandas
import os
import re
import json
from tqdm import tqdm

In [19]:
import numpy as np
import pandas as pd

In [29]:
# I am using mozzila so, geckodriver
# If you use chrome you can use anything
browser = webdriver.Firefox(service=Service("../resources/geckodriver-v0.31.0-win64/geckodriver.exe"))
browser2 = webdriver.Firefox(service=Service("../resources/geckodriver-v0.31.0-win64/geckodriver.exe"))
actions = ActionChains(browser)

In [11]:
# find all Catefory 0
browser.get("https://blinkit.com/cn/dairy-bread-eggs/milk/cid/14/922")  #give an initial site, go to browser and choose location
soup = BeautifulSoup(browser.page_source,'html.parser')
category_0 = {}
for i in soup.find_all('a', {"class": "cat-0-list__item--name"}):
    category_0.update({i.text:i['href']})

In [12]:
# find all the sub links
sub_cats = []
for k,v in tqdm(category_0.items()):
    browser.get("https://blinkit.com"+v)
    time.sleep(3) #wait time can be given according to internetspeed
    soup = BeautifulSoup(browser.page_source,"html.parser")
    all_subs = soup.find_all('li', {"class":'category-list__item no-link no-hover no-child'})
    for i in all_subs:
        sub_cats.append({k:{i.text:i.find('a')['href']}})

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [02:01<00:00,  4.67s/it]


In [30]:
with open('products.json','w') as f:
    pass

for cat in tqdm(sub_cats):    
    browser.get("https://blinkit.com"+list(list(cat.values())[0].values())[0])
    i=0
    delay = 10
    #scrolling down until all products show up
    while(i<int(browser.execute_script("return document.body.scrollHeight"))):
        i+=20 #this can be tweaked
        time.sleep(0.15) #this time can be given according to internet speed
        browser.execute_script(f"window.scrollTo(0, {i});")
    products = browser.find_elements(by = By.CLASS_NAME,value='product__wrapper')
    for product in products:
        try:
            name = product.find_element(by = By.CLASS_NAME, value="Product__ProductName-sc-11dk8zk-4").text
        except:
            name = "name not found"
        try:
            mrp = product.find_element(by = By.CLASS_NAME, value="ProductPrice__Mrp-sc-14194u2-2").text
        except:
            mrp = "mrp not found"
        try:
            price = product.find_element(by = By.CLASS_NAME, value="ProductPrice__Price-sc-14194u2-1").text
        except:
            price = "price not found"
        try:
            quantity = product.find_element(by = By.CLASS_NAME, value="plp-product__quantity--box").text
        except:
            quantity = "Quantity Not Found"
        try:
            off = product.find_element(by = By.CLASS_NAME, value="plp-product__offer").text
        except:
            off = "off not found"
        try:
            img_source = product.find_element(by = By.TAG_NAME, value = "img").get_attribute('src')
        except:
            img_source = 'problem loading url'
        try:
            product_link = product.get_attribute('href')
        except:
            product_link = 'product link not found'
        try:
            browser2.get(product_link)
            product = WebDriverWait(browser2, delay).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
            soup = BeautifulSoup(browser2.page_source, 'html.parser')
            details = dict(zip ([i.text for i in soup.find_all('span')[3:]],[i.text for i in soup.find_all('div',attrs={'class' : "ProductAttribute__ProductAttributesDescription-sc-dyoysr-2 lnLDYa"})]))
        except:
            details = 'Unable to Extract'
            
        final = {'Category':list(cat.keys())[0],'Sub_Category':list(list(cat.values())[0].keys())[0],'product_name':name,'mrp':mrp,'price':price,'quantity':quantity,'off':off,'img_source':img_source,"product_link":product_link,'details':details}
#         print(final)
        with open("products.json",'a') as f:
            json.dump(final,f)
            

100%|██████████████████████████████████████████████████████████████████████████████| 248/248 [4:01:37<00:00, 58.46s/it]


In [31]:
with open("products.json",'r') as f:
    a= f.readlines()

In [32]:
records = a[0].replace("}{","}=={").split("==")

In [33]:
vals = [list(eval(v).values()) for v in records]

In [34]:
pd.DataFrame(vals,columns=list(eval(records[0]).keys())).to_csv('products.csv',index=False)

In [35]:
df = pd.DataFrame(vals,columns=list(eval(records[0]).keys()))

In [37]:
df

,Category,Sub_Category,product_name,mrp,price,quantity,off,img_source,product_link,details
0,Vegetables & Fruits,Exotics & Premium,Plum Imported,₹251,₹198,500 g,21% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/plum-imported/prid/294787,"{'Shelf Life': '3 days', 'Manufacturer Details..."
1,Vegetables & Fruits,Exotics & Premium,Dried Apricots,₹352,₹281,125 g,20% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/dried-apricots/prid/45...,"{'Shelf Life': '3 months', 'Manufacturer Detai..."
2,Vegetables & Fruits,Exotics & Premium,Green Zucchini,₹100,₹80,1 piece (200 g - 250 g),20% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/green-zucchini/prid/25...,"{'Shelf Life': '2 days', 'Manufacturer Details..."
3,Vegetables & Fruits,Exotics & Premium,Iceberg Lettuce,₹99,₹77,250 g,22% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/iceberg-lettuce/prid/4...,"{'Shelf Life': '2 days', 'Manufacturer Details..."
4,Vegetables & Fruits,Exotics & Premium,Cherry Tomato,₹142,₹112,200 g,21% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/cherry-tomato/prid/440229,"{'Shelf Life': '2 days', 'Manufacturer Details..."
...,...,...,...,...,...,...,...,...,...,...
4001,Beauty,Shampoos & Conditioners,Wow Skin Science Red Onion Black Seed Oil Shampoo,mrp not found,₹249,200 ml,off not found,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/wow-skin-science-red-o...,"{'Shelf Life': '24 months', 'Manufacturer Deta..."
4002,Beauty,Shampoos & Conditioners,Wow Skin Science Red Onion Black Seed Onion Oi...,₹349,₹311,100 ml,10% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/wow-skin-science-red-o...,"{'Shelf Life': '24 months', 'Manufacturer Deta..."
4003,Beauty,Shampoos & Conditioners,Plum Olive & Macadamia Healthy Hydration Shampoo,₹550,₹385,300 ml,30% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/plum-olive-macadamia-h...,"{'Shelf Life': '24 months', 'Manufacturer Deta..."
4004,Beauty,Shampoos & Conditioners,Wow Skin Science Apple Cider Vinegar Made With...,₹999,₹600,1 l,39% OFF,"https://cdn.grofers.com/cdn-cgi/image/f=auto,f...",https://blinkit.com/prn/wow-skin-science-apple...,"{'Shelf Life': '24 Months', 'Manufacturer Deta..."
